In [1]:
from NeuralNetwork import NeuralNet
import jax.numpy as np
import numpy as onp


def simple_poly(X: np.ndarray) -> np.ndarray:
    return 4 - 3 * X + X**2

/Users/augustfemtehjell/opt/miniconda3/envs/fysstk/lib/python3.11/site-packages/jax/_src/api_util.py:172: SyntaxWarning: Jitted function has static_argnums=(1,), but only accepts 1 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "


In [2]:
onp.random.seed(2023)

n_points = 2000
x = onp.random.randn(n_points, 1)
y = simple_poly(x) + onp.random.randn(n_points, 1)

In [3]:
from Schedules import Constant
from jax import jit

dimension = (1, 5, 5, 5, 1)
scheduler = Constant(0.01)
FFNN = NeuralNet(dimension, seed=2023, output_func=jit(lambda x: x))

scores = FFNN.fit(x, y, scheduler, batches=25, epochs=400, lmbda=0.2)

I0000 00:00:1699804668.533830       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
  0%|          | 0/10000 [00:00<?, ?it/s]

(80, 1)


TypeError: Gradient only defined for scalar-output functions. Output had shape: (80,).

In [ ]:
import matplotlib.pyplot as plt

xnew = np.linspace(-3, 3, 100).reshape(-1, 1)
ynew = FFNN.predict(xnew)

plt.plot(x, y, "o", label="data")
plt.plot(xnew, ynew, label="prediction")
plt.plot(xnew, simple_poly(xnew), label="true")
plt.legend()
plt.show()

In [ ]:
from Schedules import Adam

dimension = (1, 5, 5, 5, 1)
scheduler = Adam(0.01, 0.9, 0.999)
FFNN = NeuralNet(dimension, seed=2023, output_func=lambda x: x)

scores = FFNN.fit(x, y, scheduler, batches=25, epochs=400, lmbda=0.2)

In [ ]:
xnew = np.linspace(-3, 3, 100).reshape(-1, 1)
ynew = FFNN.predict(xnew)

plt.plot(x, y, "o", label="data")
plt.plot(xnew, ynew, label="prediction")
plt.plot(xnew, simple_poly(xnew), label="true")
plt.legend()
plt.show()

In [ ]:
from Activators import RELU

dimension = (1, 20, 20, 20, 1)
scheduler = Adam(0.01, 0.9, 0.999)
FFNN = NeuralNet(dimension, seed=2023, output_func=lambda x: x, hidden_func=RELU)

scores = FFNN.fit(x, y, scheduler, batches=25, epochs=400, lmbda=0.2)

In [ ]:
xnew = np.linspace(-3, 3, 100).reshape(-1, 1)
ynew = FFNN.predict(xnew)

plt.plot(x, y, "o", label="data", alpha=0.2)
plt.plot(xnew, ynew, label="prediction")
plt.plot(xnew, simple_poly(xnew), label="true")
plt.legend()
plt.show()

In [ ]:
from Activators import LRELU

dimension = (1, 20, 20, 20, 1)
scheduler = Adam(0.01, 0.9, 0.999)
FFNN = NeuralNet(dimension, seed=2023, output_func=lambda x: x, hidden_func=LRELU)

scores = FFNN.fit(x, y, scheduler, batches=25, epochs=400, lmbda=0.2)

In [ ]:
xnew = np.linspace(-3, 3, 100).reshape(-1, 1)
ynew = FFNN.predict(xnew)

plt.plot(x, y, "o", label="data", alpha=0.2)
plt.plot(xnew, ynew, label="prediction")
plt.plot(xnew, simple_poly(xnew), label="true")
plt.legend()
plt.show()

In [ ]:
import autograd.numpy as np


def FrankeFunction(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    """Franke's function for evaluating methods.

    inputs:
        x (np.array): values in x direction

        y (np.array): values in y direction

    returns:
        (np.array) values in z direction
    """

    term1 = 0.75 * np.exp(-(0.25 * (9 * x - 2) ** 2) -
                          0.25 * ((9 * y - 2) ** 2))
    term2 = 0.75 * np.exp(-((9 * x + 1) ** 2) / 49.0 - 0.1 * (9 * y + 1))
    term3 = 0.5 * np.exp(-((9 * x - 7) ** 2) / 4.0 - 0.25 * ((9 * y - 3) ** 2))
    term4 = -0.2 * np.exp(-((9 * x - 4) ** 2) - (9 * y - 7) ** 2)
    return term1 + term2 + term3 + term4

In [ ]:
x = np.linspace(0, 1, 10)
y = np.linspace(0, 1, 10)
x_, y_ = np.meshgrid(x, y)
x = x_.ravel().reshape(-1, 1)
y = y_.ravel().reshape(-1, 1)

z = FrankeFunction(x, y)  # + np.random.randn(100, 100) * 0.1

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt

xnew = np.linspace(0, 1, 100)
ynew = np.linspace(0, 1, 100)
xnew, ynew = np.meshgrid(xnew, ynew)
znew = FFNN.predict(
    np.c_[xnew.ravel().reshape(-1, 1), ynew.ravel().reshape(-1, 1)]
).reshape(100, 100)

fig = plt.figure(figsize=plt.figaspect(0.5))

ax = fig.add_subplot(1, 2, 1, projection="3d")
surf = ax.plot_surface(
    x_,
    y_,
    z,
    cmap=cm.coolwarm,
    linewidth=0,
    antialiased=False,
)

# Customize the z axis.
ax.set_zlim(-0.10, 1.40)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter("%.02f"))
ax.set_title("Input data")

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=10)

ax = fig.add_subplot(1, 2, 2, projection="3d")
surf = ax.plot_surface(
    xnew,
    ynew,
    znew,
    cmap=cm.coolwarm,
    linewidth=0,
    antialiased=False,
)

# Customize the z axis.
ax.set_zlim(-0.10, 1.40)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter("%.02f"))
ax.set_title("Output data")

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=10)

fig.suptitle("Franke's function")

plt.tight_layout()

In [ ]:
from Activators import RELU

x = np.linspace(-2, 5, 1000).reshape(-1, 1)
y = simple_poly(x)


dimension = (1, 25, 1)
scheduler = Adam(0.01, 0.9, 0.999)
FFNN = NeuralNet(dimension, seed=2023, output_func=lambda x: x, hidden_func=RELU)

scores = FFNN.fit(x, y, scheduler, batches=25, epochs=200, lmbda=0.2)

In [ ]:
ypred = FFNN.predict(x)

plt.scatter(x, y, label="data", alpha=0.2)
plt.plot(x, ypred, label="prediction")
plt.legend()
plt.show()

In [ ]:
ypred.shape

In [ ]:
accuracy = np.array(scores["train_errors"])
t = np.arange(accuracy.shape[0])
plt.plot(t, accuracy[t])

In [ ]:
scores.keys()

In [ ]:
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
x_, y_ = np.meshgrid(x, y)
x = x_.ravel().reshape(-1, 1)
y = y_.ravel().reshape(-1, 1)

In [ ]:
def design_poly(x, y, dim):
    x = x.ravel()
    y = y.ravel()
    X = np.zeros((x.shape[0], dim * (dim + 1) // 2))
    k = 0
    for i in range(dim):
        for j in range(i + 1):
            X[:, k] = x ** (i - j) * y**j
            k += 1
    return X


X = design_poly(x, y, 10)
X.shape

In [ ]:
from Activators import sigmoid
from sklearn.preprocessing import StandardScaler

z = FrankeFunction(x, y)  # + np.random.randn(100, 100) * 0.1
scaler = StandardScaler()
z = scaler.fit_transform(z)
scheduler = Adam(0.01, 0.9, 0.999)
FFNN = NeuralNet((55, 55, 1), seed=2023,
                 output_func=lambda x: x, hidden_func=sigmoid)
scores = FFNN.fit(X, z, scheduler, batches=25, epochs=400, lmbda=0.2)

In [ ]:
plt.plot(scores["train_errors"][10:])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt

znew = FFNN.predict(X)

fig = plt.figure(figsize=plt.figaspect(0.5))

ax = fig.add_subplot(1, 2, 1, projection="3d")
surf = ax.plot_surface(
    x_,
    y_,
    FrankeFunction(x_, y_),
    cmap=cm.coolwarm,
    linewidth=0,
    antialiased=False,
)

# Customize the z axis.
ax.set_zlim(-0.10, 1.40)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter("%.02f"))
ax.set_title("Input data")

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=10)

ax = fig.add_subplot(1, 2, 2, projection="3d")
surf = ax.plot_surface(
    x_,
    y_,
    znew.reshape(100, 100),
    cmap=cm.coolwarm,
    linewidth=0,
    antialiased=False,
)

# Customize the z axis.
ax.set_zlim(-0.10, 1.40)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter("%.02f"))
ax.set_title("Output data")

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=10)

fig.suptitle("Franke's function")

plt.tight_layout()